Setup guide: Shift to readme

In [1]:
%%bash

cat <<EOL > setup_env.sh
source ~/env_MPI/bin/activate
export PYTHONPATH=~/env_MPI/lib/python3.11/site-packages:\$PYTHONPATH
module load python3/3.11.0
module load openmpi/4.0.5

export NPrt=24
export WorkDir=~/PCG_MPI
cd \$WorkDir
EOL


In [2]:
%%bash
source setup_env.sh

export ModelName=CubeTest
export ScratchPath=$WorkDir/$ModelName
export InputModelPath=$ScratchPath/$ModelName.zip

python3 ReadInputModel.py $WorkDir $ModelName $ScratchPath $InputModelPath


In [3]:
%%bash
source setup_env.sh

python3 MetisMeshPart.py $NPrt



N_Parts:  24


In [4]:
%%bash
source setup_env.sh

export NSplits=4   
mpiexec -np $NSplits --map-by numa python3 Elast_ParDataPrepMPI.py $NPrt 0


N_Workers 4


0 A


0 B0


0 B1


0 C


...

0 G


0 H


0 I


0 J


0 K


In [5]:
%%bash
source setup_env.sh

python3 -c "
from GeneralFunc import exportz
TimeHistoryParam={'ExportFlag': True, # export results
                  'ExportFrmRate' : 1,
                  'ExportFrms' : [],
                  'PlotFlag' : False,
                  'TimeStepDelta': [0,1],
                  'ExportVars': 'U'} #D U PS PE GS GE

SolverParam={'Tol' : 1e-7,
             'MaxIter' : 10000}

GlobSettings = {'TimeHistoryParam':TimeHistoryParam,
                'SolverParam':SolverParam}
exportz('__pycache__/GlobSettings.zpkl', GlobSettings)
"

In [6]:
%%bash
source setup_env.sh

mpiexec -np $NPrt --map-by numa python3 Elast_StaticAnalysis.py 1 0


N_Workers 24


Time (sec) taken to read files:  0.1


0 1.0 [0.6 0. ]


100 1.3422084271370702e-05 [0.6 0. ]


Analysis Finished Sucessfully..


TotalTimeStepCount 2


-----------------


Verify Total Time: 0.04721488160982809 0.23130341370900476


Total Time: 0.9024715423583984 ['Mean', 0.8987173438072205] ['Max_i', 0.9023511409759521]


In [7]:
%%bash
source setup_env.sh

mpiexec -np 1 python3 ExportVTK.py 1 "U" "Full"


0


1
